# TP3 : Super-héros et Sudoku

In [1]:
# Librairies à importer pour utiliser JuMP avec le solver GLPK
using JuMP
using GLPK

# Définition de constantes pour le statut de résolution du problème
const OPTIMAL = JuMP.MathOptInterface.OPTIMAL
const INFEASIBLE = JuMP.MathOptInterface.INFEASIBLE
const UNBOUNDED = JuMP.MathOptInterface.DUAL_INFEASIBLE;

## Exercice 1 : Équipe de superhéros

Pour combattre les aliens qui envahissent la terre, il faut créer une équipe de superhéros travaillant main dans la main. Malheureusement, certains superhéros sont ennemis et ne peuvent donc pas faire équipe...  Combien de superhéros peuvent aller combattre les aliens ? Il faut trouver l'équipe la plus importante possible sans ennemis... Le sort de la terre en dépend !

Voici la liste des superhéros : 
   * Batman
   * Superman
   * Catwoman
   * Flash
   * Wonder woman
   * Black Panther
   * Captain America
   * Daredevil
   * Elektra
   * Hulk

Et voici maintenant la liste des ennemis jurés : 
   * Batman et Flash
   * Catwoman et Captain America
   * Daredevil et Elektra
   * Hulk et Batman
   * Catwoman et Wonder woman
   * Black Panther et Hulk
   * Superman et Flash
   * Superman et Elektra
   * Flash et Daredevil
   * Wonder woman et Captain America
   * Daredevil et Hulk
   * Batman et Captain America
   * Batman et Wonder woman
   * Black Panther et Wonder woman



**Remarque :** Il est possible de définir des variables indexées par un tableau de chaînes de caractères. Par exemple, il est possible de déclarer les variables `@variable(m,x[Superhéros])`





In [2]:
#=
    DONNÉES
=#

Superhéros = [
 "Batman",
 "Superman",
 "Catwoman",
 "Flash",
 "Wonder woman",
 "Black Panther",
 "Captain America",
 "Daredevil",
 "Elektra",
 "Hulk"
]


Ennemis = [
  ["Batman", "Flash"],
  ["Catwoman", "Captain America"],
  ["Daredevil", "Elektra"],
  ["Hulk", "Batman"],
  ["Catwoman", "Wonder woman"],
  ["Black Panther", "Hulk"],
  ["Superman", "Flash"],
  ["Superman", "Elektra"],
  ["Flash", "Daredevil"],
  ["Wonder woman", "Captain America"],
  ["Daredevil", "Hulk"],
  ["Batman", "Captain America"],
  ["Batman", "Wonder woman"],
  ["Black Panther", "Wonder woman"]
];

In [17]:
############################## 
#   Saisir votre code ici.   #
##############################
function find_superhero()
    m = Model(GLPK.Optimizer)

    @variable(m,x[Superhéros], Bin)

    @objective(m, Max, sum(x[super] for super in Superhéros))

    for truc in Ennemis
        @constraint(m, x[truc[1]]+x[truc[2]]<=1)
    end

    print(m)

    JuMP.optimize!(m)

    println("Heros can save the world is : ", objective_value(m))

    #Affichage du résultat
    println("List of the equipe")
    count = 0
    for super in Superhéros
        if JuMP.value(x[super])==1
            count = count + 1
            println("Hero $count : $super")
        end
    end
end

find_superhero (generic function with 1 method)

In [18]:
find_superhero()

Heros can save the world is : 5.0
List of the equipe
Hero 1 : Batman
Hero 2 : Superman
Hero 3 : Catwoman
Hero 4 : Black Panther
Hero 5 : Daredevil


## Exercice 2 : Pause sudoku

Le sudoku est une grille carrée de taille $9\times 9$ divisée en 9 régions carrées de 9 cases. On doit remplir chaque case avec un chiffre de 1 à 9 en respectant la régle suivante : dans chaque ligne, chaque colonne, chaque région, les chiffres de 1 à 9 apparaissent une et une seule fois. Certaines cases sont déjà préremplies.

Le but de cet exercice est de résoudre la grille de sudoku ci-dessous **à l'aide de la programmation linéaire en nombres entiers !**

Formuler le problème du sudoku comme un programme linéaireen nombres entiers, et résoudre la grille suivante (les zeros représentent les cases à remplir).

**Indication :** On utilisera comme variable `x[i,j,k]` qui vaut 1 si la case (i,j) contient la valeur k, et 0 sinon.

In [155]:
#=
    DONNÉES
=#

# les zeros représentent les cases non remplies
Grille = [
  5 3 0 0 7 0 0 0 0;
  6 0 0 1 9 5 0 0 0;
  0 9 8 0 0 0 0 6 0;
  8 0 0 0 6 0 0 0 3;
  4 0 0 8 0 3 0 0 1;
  7 0 0 0 2 0 0 0 6;
  0 6 0 0 0 0 2 8 0;
  0 0 0 4 1 9 0 0 5;
  0 0 0 0 8 0 0 7 9
];
# start from 1

In [160]:
############################## 
#   Saisir votre code ici.   #
##############################

function solveSudoku(Grille)
    sudoku = Model(GLPK.Optimizer)
    @variable(sudoku, x[1:9,1:9, 1:9], Bin)

    # only 1 valeur k in each case => = 1, the rest = 0
    for i in 1:9 #col
        for j in 1:9 #row
            @constraint(sudoku, sum(x[i,j,k] for k in 1:9)==1)
        end
    end

    # number show in row can be only 1
    # vd: row 1 have only one number 3
    for i in 1:9
        for k in 1:9
            @constraint(sudoku, sum(x[i,j,k] for j in 1:9)==1)
        end
    end

    # number show in col can be only 1
    # vd: row 1 have only one number 3
    for j in 1:9
        for k in 1:9
            @constraint(sudoku, sum(x[i,j,k] for i in 1:9)==1)
        end
    end

    # loop, take left part 0 1 2
    for i in 0:2, j in 0:2
        for k in 1:9
            @constraint(sudoku, sum(x[3*i+1:3*i+3,3*j+1:3*j+3,k])==1)
        end
    end

    # remplir table
    for i in 1:9, j in 1:9
        if Grille[i,j]!=0
           @constraint(sudoku, x[i, j, Grille[i,j]] == 1) 
        end
    end

    #print(m)

    JuMP.optimize!(sudoku)
    status = termination_status(sudoku)
    println("Status is : ", status)

    # affichage resultat

    Solution = Grille
    #print(JuMP.value(x[1,1,3]))
    for i in 1:9
        for j in 1:9
           if Solution[i,j]==0
                for k in 1:9
                    #println(sum(JuMP.value(x[i,j,k]*k for k in 1:9)))
                    Solution[i,j] = Solution[i,j] + JuMP.value(x[i,j,k]*k)
                end
            end
        end
    end

    # show more pretty
    println()
    for i in 1:9
        for j in 1:9
            print(Solution[i,j], " ")
        end
        println()
    end
end


solveSudoku (generic function with 1 method)

In [161]:
solveSudoku(Grille)

Status is : OPTIMAL

5 3 4 6 7 8 9 1 2 
6 7 2 1 9 5 3 4 8 
1 9 8 3 4 2 5 6 7 
8 5 9 7 6 1 4 2 3 
4 2 6 8 5 3 7 9 1 
7 1 3 9 2 4 8 5 6 
9 6 1 5 3 7 2 8 4 
2 8 7 4 1 9 6 3 5 
3 4 5 2 8 6 1 7 9 
